In [1]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, 'bin')

from persistence import sql
import functional as fn
import color as c

print(fn.build('python','length', ['name']))
print(fn.build('programming','call',['len','name']))
print(fn.build('sql','left join'))



len(name)

LEFT JOIN
  {{table}} ON {{table}}.{{identifier}} = {{subject}}.{{alias}}



In [9]:
fn.create_command('dotnet', 'length','',['messurable'])
fn.set_child(('python', 'length'), ('programming', 'call'),{'function': 'len','parameter':'{{messurable}}'})

In [ ]:
fn.create_command('myapp', 'conditional query')
fn.set_child(('myapp', 'conditional query'),('python','if'),{'block':('myapp','select person')})

In [ ]:
db = sql('functional')

for i in db.query('SELECT * FROM command'):
    print(c.blue + i['class'] + ':' + i['name'])
    print(c.white + fn.build(i['class'], i['name']))

db.close()

In [2]:
print(fn.build('sql', 'select from', 
{
    'field': '*',
    'table': 'usuario'
})+
fn.build('sql','left join',{
    'table': 'pessoa',
    'identifier': 'id',
    'subject': 'usuario',
    'alias': 'id_pessoa'
}))

SELECT
  *
FROM
  usuario
LEFT JOIN
  pessoa ON pessoa.id = usuario.id_pessoa



In [ ]:
fn.create_command('python','if','''if({{condition}}):
{{block}}''')

In [ ]:
fn.create_command('programming', 'proprety','{{parent}}.{{child}}',['value'])


In [ ]:
print(fn.build('sql','select from',{}))

In [ ]:
cur = '''SELECT
    {{fields}}
FROM
    {{table}}
    {{junk}}'''

temp = cur

parameter = [
    {'parameter':'{{fields}}','value':'''id,
nome,
idade'''},
    {'parameter':'{{table}}','value':'pessoa'},
    {'parameter':'{{junk}}','value':'aaa'}
]
cur_parameter = parameter.copy()
j = 0
spaces = 0
start = 0
diference = 0
for i in range(len(cur)):
    found = False
    for k in cur_parameter:
        if(j >= len(k['parameter'])-1):
            value = k['value']
            padding = ' ' * (spaces)
            value = padding + value.replace('\n','\n'+padding)
            temp = temp[0:start-spaces+1] + value + temp[i+1-diference:len(temp)]
            diference += j + 2 - len(value)+spaces-1
            found = True
            break
    if(not found):
        cur_parameter = [k for k in cur_parameter if k['parameter'][j] == hay[i]]
        j += 1
    if(len(cur_parameter) == 0 or found):
        if hay[i] == ' ':
            spaces += 1
        else:
            spaces = 0
        j = 0
        start = i - diference
        cur_parameter = parameter.copy()

print (temp)